# Sea Surface Temperature

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from maps import map_robinson
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl
from regions import SOM_area, Nino12, Nino34
from plotting import shifted_color_map
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global SST map and trends

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

In [ ]:
# %%time
# # stacking files into one xr Dataset object
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'yrly', name='TEMP_PD')):
#         print(y)
#         da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         da = da.drop(['z_t', 'ULONG', 'ULAT'])
#         da['TLAT' ] = da['TLAT' ].round(decimals=2)
#         da['TLONG'] = da['TLONG'].round(decimals=2)
#         del da.encoding["contiguous"]
#         ds = t2ds(da=da, name='SST', t=int(round(da.time.item())))
#         ds.to_netcdf(path=f'{path_samoc}/SST/SST_yrly_{run}_{y}.nc', mode='w')
     
#     combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_yrly_{run}_*.nc',
#                                  concat_dim='time',
#                                  autoclose=True,
#                                  coords='minimal')
#     combined.to_netcdf(f'{path_samoc}/SST/SST_yrly_{run}.nc')
#     # remove extra netCDF files

CPU times: user 1min 25s, sys: 3min 16s, total: 4min 41s
Wall time: 11min 18s

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )

In [ ]:
%%time
SST_trend_ctrl = ocn_field_regression(SST_yrly_ctrl)
SST_trend_rcp  = ocn_field_regression(SST_yrly_rcp )

In [ ]:
fn = f'{path_results}/SST/SST_ctrl_mean'
xa = SST_yrly_ctrl.mean(dim='time').where(MASK>0)
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmocean.cm.thermal, minv=-2, maxv=30, label='SST [$^\circ$C]', filename=fn)

In [ ]:
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'

xa = SST_trend_ctrl*100
fn = f'{path_results}/SST/SST_trend_ctrl'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

xa = SST_trend_rcp*100
fn = f'{path_results}/SST/SST_trend_rcp'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

### hiatus trend

In [ ]:
SST_yrly_rcp[20:29,:,:]

In [ ]:
SST_trend_rcp_hiatus  = ocn_field_regression(SST_yrly_rcp[20:29,:,:])

In [ ]:
fn = f'{path_results}/SST/SST_trend_rcp_hiatus'
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'
xa = SST_trend_rcp_hiatus*100
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-4, maxv=8, label=label, filename=fn)

# SST indices

In [ ]:
TAREA     = xr_AREA('ocn')
AREA_rect = xr_AREA('ocn_rect')
AREA_rect = AREA_rect.rename({'t_lon':'lon', 't_lat':'lat'})

In [ ]:
def SST_index(xa_SST, AREA, index_loc, AREA_index):
    """ calculates the average SST over an area, possibly as a time series """
    assert type(xa_SST)==xr.core.dataarray.DataArray
    assert type(AREA)==xr.core.dataarray.DataArray
    assert type(index_loc)==dict
#     for dim in ['nlat', 'nlon']:
#         for xa in [xa_SST, AREA]:
#             print(dim,xa)
#             assert dim in xa

    index = (xa_SST*AREA).sel(index_loc).sum(dim=('nlat', 'nlon'))/AREA_index
    return index

# Southern Ocean
## Southern Ocean Mode index

In [ ]:
SOMi_area

In [ ]:
%%time
SOMi_area = TAREA.sel(SOM_area).sum()
SOMi_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA, index_loc=SOM_area, AREA_index=SOMi_area)
SOMi_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA, index_loc=SOM_area, AREA_index=SOMi_area)

In [ ]:
SOMi_trend_ctrl = np.polyfit(SOMi_ctrl.time, SOMi_ctrl, 1)
SOMi_trend_rcp  = np.polyfit(SOMi_rcp .time, SOMi_rcp , 1)

SOMi_detr_ctrl = SOMi_ctrl - (SOMi_trend_ctrl[1] + SOMi_trend_ctrl[0]*SOMi_ctrl.time)
SOMi_detr_rcp  = SOMi_rcp  - (SOMi_trend_rcp[1]  + SOMi_trend_rcp[0] *SOMi_rcp .time)

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

# plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp , c='C1', lw=.5)

plt.plot(SOMi_ctrl.time/365     , SOMi_trend_ctrl[1] + SOMi_trend_ctrl[0]*SOMi_ctrl.time, ls='--')
plt.plot(SOMi_rcp .time/365-1800, SOMi_trend_rcp [1] + SOMi_trend_rcp [0]*SOMi_rcp.time , ls='--')

L1, = plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
L2, = plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2, loc=4, handles=[L1, L2])
plt.savefig(f'{path_results}/SST/SOM_raw')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp , c='C1', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
plt.savefig(f'{path_results}/SST/SOM_index')

# Pacific Ocean

## Interdecadal Pacific Oscillation

## Pacific decadal oscillation
"The leading EOF of monthlySST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)"

## ENSO

In [ ]:
N12_ctrl = ((SST_yrly_ctrl*TAREA).sel(Nino12).sum(dim=('nlat','nlon'))/TAREA.sel(Nino12).sum(dim=('nlat','nlon')))
N34_ctrl = ((SST_yrly_ctrl*TAREA).sel(Nino34).sum(dim=('nlat','nlon'))/TAREA.sel(Nino34).sum(dim=('nlat','nlon')))
N12_rcp  = ((SST_yrly_rcp *TAREA).sel(Nino12).sum(dim=('nlat','nlon'))/TAREA.sel(Nino12).sum(dim=('nlat','nlon')))
N34_rcp  = ((SST_yrly_rcp *TAREA).sel(Nino34).sum(dim=('nlat','nlon'))/TAREA.sel(Nino34).sum(dim=('nlat','nlon')))

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
ds.TEMP

In [ ]:
# %%time
# #5-10 sec per year
# Nino12_AREA = TAREA.sel(Nino12).sum(dim=('nlat','nlon'))
# Nino34_AREA = TAREA.sel(Nino34).sum(dim=('nlat','nlon'))
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         if m==1: print(y)
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         N12 = SST_index(xa_SST=SST, AREA=TAREA, index_loc=Nino12, AREA_index=Nino12_AREA)
#         N34 = SST_index(xa_SST=SST, AREA=TAREA, index_loc=Nino34, AREA_index=Nino34_AREA)
#         if i==0:
#             N12_new = N12
#             N34_new = N34
#         else:
#             N12_new = xr.concat([N12_new, N12], dim='time')
#             N34_new = xr.concat([N34_new, N34], dim='time')
#     N12_new.to_netcdf(path=f'{path_samoc}/SST/Nino12_{run}.nc')
#     N34_new.to_netcdf(path=f'{path_samoc}/SST/Nino34_{run}.nc')

In [ ]:
N12_ctrl = xr.open_dataarray(f'{path_samoc}/SST/Nino12_ctrl.nc', decode_times=False)
N34_ctrl = xr.open_dataarray(f'{path_samoc}/SST/Nino34_ctrl.nc', decode_times=False)
N12_rcp  = xr.open_dataarray(f'{path_samoc}/SST/Nino12_rcp.nc' , decode_times=False)
N34_rcp  = xr.open_dataarray(f'{path_samoc}/SST/Nino34_rcp.nc' , decode_times=False)

In [ ]:
N12_rcp

In [ ]:
N12_ctrl

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(N12_ctrl.time/365    , N12_ctrl.rolling({'time':12}, center=True).mean())
plt.plot(N34_ctrl.time/365    , N34_ctrl.rolling({'time':12}, center=True).mean())
plt.plot(N12_rcp.time/365-1800, N12_rcp .rolling({'time':12}, center=True).mean())
plt.plot(N34_rcp.time/365-1800, N34_rcp .rolling({'time':12}, center=True).mean())
plt.tight_layout()

# Atlantic Ocean
## Atlantic Multidecadal Oscillation